In [1]:
using DiffEqGPU, OrdinaryDiffEq, CUDA
using BenchmarkTools
using StaticArrays
include("./CUDAInterpolations.jl")
using .CUDAInterpolations

In [2]:
@inline function linspace(a, b, N=100)
    return (b-a) * (0:(N-1)) / N .+ a
end

function background_U!(U, N, x, t)
    @. U = 0.2 + 0.05 * cos(π*t) * sin.(x)
end

function background_Ux!(Ux, N, x, t)
    @. Ux = 0.05 * cos(π*t) * cos.(x)
end

function background_Uxx!(Uxx, N, x, t)
    @. Uxx = -0.05 * cos(π*t) * sin.(x)
end

function bind_broadcast_interpolate(U1, U2, Ux1, Ux2, tspan, nodes, N, dx)
    u1(x) = cubic_interpolate(x, U1, Ux1, nodes, dx, N)
    u2(x) = cubic_interpolate(x, U2, Ux2, nodes, dx, N)
    
    function broadcast_interpolate(x, t)
        u1_vals = map(u1, x)
        u2_vals = map(u2, x)
        return two_point_linear_interpolate(t, u1_vals, u2_vals, tspan[1], tspan[2])
    end
    return broadcast_interpolate
end

function bind_interpolate(U1, U2, Ux1, Ux2, tspan, nodes, N, dx)
    function interpolate(x, t)
        u1 = cubic_interpolate(x, U1, Ux1, nodes, dx, N)
        u2 = cubic_interpolate(x, U2, Ux2, nodes, dx, N)
        return two_point_linear_interpolate(t, u1, u2, tspan[1], tspan[2])
    end
end

bind_interpolate (generic function with 1 method)

In [3]:
N = 128
Lx = 2π
x = CuArray(convert.(Float32, linspace(0, Lx, N)))
dx = N / Lx
tspan = (0, 1)

U1   = CuArray{Float32}(undef, N)
U1x  = CuArray{Float32}(undef, N)
U1xx = CuArray{Float32}(undef, N)
U2   = CuArray{Float32}(undef, N)
U2x  = CuArray{Float32}(undef, N)
U2xx = CuArray{Float32}(undef, N)
background_U!(  U1,   N, x, tspan[1])
background_Ux!( U1x,  N, x, tspan[1])
background_Uxx!(U1xx, N, x, tspan[1])
background_U!(  U2,   N, x, tspan[2])
background_Ux!( U2x,  N, x, tspan[2])
background_Uxx!(U2xx, N, x, tspan[2])

t = 0.125
xs = 2π * CUDA.rand(1000)
interpolator = bind_broadcast_interpolate(U1, U2, U1x, U2x, tspan, x, N, dx)
@btime interpolator(xs, t)

  65.167 μs (485 allocations: 10.94 KiB)


1000-element CuArray{Float64, 1, CUDA.DeviceMemory}:
 0.27070017546097064
 0.2723886072763087
 0.2724553507067346
 0.26570681248555195
 0.26725015402420155
 0.27361865640581934
 0.27184304531207365
 0.2576559350808809
 0.2701266533121059
 0.2146659575431981
 0.26224666778310685
 0.26114961293197275
 0.25000721492575434
 ⋮
 0.27185003259881024
 0.2129216065320686
 0.27138917650335204
 0.2651441067326269
 0.257112849714967
 0.2556487499059167
 0.2021280397752272
 0.26964311754314224
 0.21888667488741134
 0.2520152987559264
 0.23090035835336342
 0.2566547038463808

In [16]:
using StaticArrays

function ω(k, p)
    return sqrt(p.f^2 + p.Cg^2*k^2)
end

function group_velocity(k, p)
    return p.Cg^2*k/ω(k, p)
end

function raytrace(dxk, xk, params, t)
    group_vel = group_velocity(xk[2], params)
    dxk[1] = params.u(xk[1], t) + group_vel;
    dxk[2] = -params.ux(xk[1], t) * xk[2]
end

function raytrace1_5(xk, params, t)
    return SVector{2}(1, 1)
end

function raytrace2(xk, params, t)
    x = xk[1]
    k = xk[2]
    x_normalized = (x - params.x0) / params.Lx
    group_vel = group_velocity(k, params)
    u1 = params.U1
    #u1x = params.U1x[x_normalized]
    #u2 = params.U2[x_normalized]
    #u2x = params.U2x[x_normalized]
    #u  = two_point_linear_interpolate(t, u1,  u2,  params.t0, params.t1)
    #ux = two_point_linear_interpolate(t, u1x, u2x, params.t0, params.t1)
    #dx = u + group_vel;
    #dk = -ux * k
    return SVector{2}(0., 0.)
end

function dxdt(xdot, x, k, params, t)
    group_vel = group_velocity(k, params)
    xdot[1] = params.u(x[1], t) + group_vel[1];
end

function dkdt(kdot, x, k, params, t)
    kdot[1] = -params.ux(x[1], t) * k[1]
end

function AB1(f, dt, t, y)
    return y + dt * f(t, y)
end

function AB2(f, dt, t, y, y_prev)
    return y + t*(3*f(t, y) - f(t

function AB3(f, dt, t, y, y_prev1, y_prev2)
    
end

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///home/nad9961/JuliaRaytracingSW/In[16]#53:1\[90mIn[16]:53:1[0;0m]8;;\

[48;2;120;70;70mfunction AB3(f, dt, t, y, y_prev1, y_prev2[0;0m)
[90m└────────────────────────────────────────┘ ── [0;0m[91mExpected `)`[0;0m

In [5]:
xk0S = @SVector Float32[1.0f0; 1.0f0]
xk0 = Float32[1.0f0; 1.0f0]
tspan = (0.0f0, 1.0f0)
params = (u  = bind_interpolate(U1,  U2,  U1x,  U2x,  tspan, x, N, dx),
          ux = bind_interpolate(U1x, U2x, U1xx, U2xx, tspan, x, N, dx),
          f  = 1.0f0,
          Cg = 1.0f0);
#prob = DynamicalODEProblem(dxdt, dkdt, x0, k0, tspan, params)
N_trajectories=1_000
xk0_array = rand(Float32, N_trajectories, 2)

1000×2 Matrix{Float32}:
 0.672289   0.808696
 0.0172395  0.736838
 0.384553   0.000753164
 0.827969   0.750407
 0.113672   0.202352
 0.981669   0.693599
 0.657712   0.315232
 0.699367   0.0142295
 0.823105   0.158962
 0.110966   0.609824
 0.395789   0.101469
 0.902849   0.687438
 0.913886   0.0715594
 ⋮          
 0.167568   0.613026
 0.7        0.86874
 0.475178   0.506129
 0.802863   0.104699
 0.929056   0.894391
 0.570592   0.270123
 0.0686752  0.649342
 0.629235   0.757455
 0.843742   0.99679
 0.738429   0.445779
 0.78896    0.210621
 0.163801   0.925872

In [6]:
params1 = (u  = bind_interpolate(U1,  U2,  U1x,  U2x,  tspan, x, N, dx),
          ux = bind_interpolate(U1x, U2x, U1xx, U2xx, tspan, x, N, dx),
          f  = 1.0f0,
          Cg = 1.0f0);
prob_func1 = (prob, i, repeat) -> remake(prob, u0=xk0_array[i, :])
prob1 = ODEProblem(raytrace, xk0, tspan, params1, dt=0.01)
ens_prob1 = EnsembleProblem(prob1, prob_func = prob_func1, safetycopy = false)
@btime sol = solve(ens_prob1, Tsit5(), EnsembleThreads(), 
    trajectories=N_trajectories, save_on = false, save_start=false);

┌ Warning: Performing scalar indexing on task Task (runnable) @0x000014e7738039e0.
│ Invocation of getindex resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore should be avoided.
│ 
│ If you want to allow scalar iteration, use `allowscalar` or `@allowscalar`
│ to enable scalar iteration globally or for the operations in question.
└ @ GPUArraysCore /ext3/pkgs/packages/GPUArraysCore/GMsgk/src/GPUArraysCore.jl:149


  6.872 s (3945861 allocations: 95.05 MiB)


In [7]:
P = 100
y = 10 * 2π * CUDA.rand(P)

texture_U1  = CuTexture(texU1;  address_mode=CUDA.ADDRESS_MODE_WRAP, interpolation=CUDA.LinearInterpolation(), normalized_coordinates=false)
texture_U1x = CuTexture(texU1x; address_mode=CUDA.ADDRESS_MODE_WRAP, interpolation=CUDA.LinearInterpolation(), normalized_coordinates=false)
function kern_get_point(output, U, idxs)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    idx = idx[i]
    output[i] = U[idx]
    return nothing
end

output = CuArray(zeros(P))
@cuda threads=(P) kern_get_point(output, texture_U1, texture_U1x, x, y)
output

LoadError: UndefVarError: `texU1` not defined

In [8]:
@inline function calcpoint(blockIdx, blockDim, threadIdx, size)
    i = (blockIdx - 1) * blockDim + threadIdx
    return i, Float32(i)
end

function kernel_texture_warp_native(dst::CuDeviceArray{<:Any,2}, texture::CuDeviceTexture{<:Any,2})
    i, u = calcpoint(blockIdx().x, blockDim().x, threadIdx().x, size(dst)[1])
    j, v = calcpoint(blockIdx().y, blockDim().y, threadIdx().y, size(dst)[2])
    @inbounds dst[i,j] = texture[2*u,2*v]
    return nothing
end

function fetch_all(texture)
    dims = size(texture)
    d_out = CuArray{eltype(texture)}(undef, dims...)

    kernel = @cuda launch=false kernel_texture_warp_native(d_out, texture)
    config = launch_configuration(kernel.fun)

    dim_x, dim_y, dim_z = size(texture, 1), size(texture, 2), size(texture, 3)
    threads_x = min(dim_x, config.threads)
    blocks_x = cld(dim_x, threads_x)

    kernel(d_out, texture; threads=threads_x, blocks=(blocks_x, dim_y, dim_z))
    d_out
end

testheight, testwidth, testdepth = 16, 16, 4
a2D = convert(Array{Float32}, repeat(1:testheight, 1, testwidth) + repeat(0.01 * (1:testwidth)', testheight, 1))
d_a2D = CuArray(a2D)

# NOTE: tex1D is not supported for linear memory

# This works as long as d_a2D is well pitched
texwrap2D = CuTexture(d_a2D; address_mode=CUDA.ADDRESS_MODE_MIRROR)
fetch_all(texwrap2D)

16×16 CuArray{Float32, 2, CUDA.DeviceMemory}:
  2.02   2.04   2.06   2.08   2.1  …   2.16   2.16   2.16   2.16   2.16
  4.02   4.04   4.06   4.08   4.1      4.16   4.16   4.16   4.16   4.16
  6.02   6.04   6.06   6.08   6.1      6.16   6.16   6.16   6.16   6.16
  8.02   8.04   8.06   8.08   8.1      8.16   8.16   8.16   8.16   8.16
 10.02  10.04  10.06  10.08  10.1     10.16  10.16  10.16  10.16  10.16
 12.02  12.04  12.06  12.08  12.1  …  12.16  12.16  12.16  12.16  12.16
 14.02  14.04  14.06  14.08  14.1     14.16  14.16  14.16  14.16  14.16
 16.02  16.04  16.06  16.08  16.1     16.16  16.16  16.16  16.16  16.16
 16.02  16.04  16.06  16.08  16.1     16.16  16.16  16.16  16.16  16.16
 16.02  16.04  16.06  16.08  16.1     16.16  16.16  16.16  16.16  16.16
 16.02  16.04  16.06  16.08  16.1  …  16.16  16.16  16.16  16.16  16.16
 16.02  16.04  16.06  16.08  16.1     16.16  16.16  16.16  16.16  16.16
 16.02  16.04  16.06  16.08  16.1     16.16  16.16  16.16  16.16  16.16
 16.02  16.04  16.

In [9]:
texU1 = CuTexture(U1;   address_mode=CUDA.ADDRESS_MODE_WRAP, interpolation=CUDA.LinearInterpolation(), normalized_coordinates=true)
texU2 = CuTexture(U2;   address_mode=CUDA.ADDRESS_MODE_WRAP, interpolation=CUDA.LinearInterpolation(), normalized_coordinates=true)
texU1x = CuTexture(U1x; address_mode=CUDA.ADDRESS_MODE_WRAP, interpolation=CUDA.LinearInterpolation(), normalized_coordinates=true)
texU2x = CuTexture(U2x; address_mode=CUDA.ADDRESS_MODE_WRAP, interpolation=CUDA.LinearInterpolation(), normalized_coordinates=true)

128-element 1-channel CuTexture(::CuArray) with eltype Float32

In [17]:
params2 = (U1 = texU1, U2 = texU2, U1x = texU1x, U2x = texU2x,
            Lx = 2π, x0 = 0., t0 = tspan[1], t1 = tspan[2],
            f = 1.0f0,
            Cg = 1.0f0)
prob_func2 = (prob, i, repeat) -> remake(prob)
prob2 = ODEProblem(raytrace2, xk0S, tspan, params2, dt=0.01)
ens_prob2 = EnsembleProblem(prob2, prob_func = prob_func2, safetycopy = false)
@btime sol = solve(ens_prob2, GPUTsit5(), EnsembleGPUKernel(CUDA.CUDABackend()),
       trajectories=N_trajectories, save_on = false, save_start=false);

  1.438 ms (45628 allocations: 2.95 MiB)


In [19]:
cudaconvert(ens_prob2)

EnsembleProblem with problem ODEProblem

In [102]:
function lorenz2(u, p, t)
    σ = p[1]
    ρ = p[2]
    β = two_point_linear_interpolate(t, p[3], p[2], 0.0f0, 10.0f0)
    du1 = σ * (u[2] - u[1])
    du2 = u[1] * (ρ - u[3]) - u[2]
    du3 = u[1] * u[2] - β * u[3]
    return SVector{3}(du1, du2, du3)
end

u0 = @SVector [1.0f0; 0.0f0; 0.0f0]
tspan = (0.0f0, 10.0f0)
p = @SVector [10.0f0, 28.0f0, 8 / 3.0f0]
prob = ODEProblem{false}(lorenz2, u0, tspan, p)
prob_func = (prob, i, repeat) -> remake(prob, p = (@SVector rand(Float32, 3)) .* p)
monteprob = EnsembleProblem(prob, prob_func = prob_func, safetycopy = false)
sol = solve(monteprob, GPUTsit5(), EnsembleGPUKernel(CUDA.CUDABackend()),
    trajectories = 10_000,
    saveat = 1.0f0)

EnsembleSolution Solution of length 10000 with uType:
ODESolution{Float32, 2, SubArray{SVector{3, Float32}, 1, Matrix{SVector{3, Float32}}, Tuple{UnitRange{Int64}, Int64}, true}, Nothing, Nothing, SubArray{Float32, 1, Matrix{Float32}, Tuple{UnitRange{Int64}, Int64}, true}, Nothing, Nothing, DiffEqGPU.ImmutableODEProblem{SVector{3, Float32}, Tuple{Float32, Float32}, false, SVector{3, Float32}, ODEFunction{false, SciMLBase.AutoSpecialize, typeof(lorenz2), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing, Nothing, Nothing}, Base.Pairs{Symbol, Union{}, Tuple{}, @NamedTuple{}}, SciMLBase.StandardODEProblem}, GPUTsit5, SciMLBase.LinearInterpolation{SubArray{Float32, 1, Matrix{Float32}, Tuple{UnitRange{Int64}, Int64}, true}, SubArray{SVector{3, Float32}, 1, Matrix{SVector{3, Float32}}, Tuple{UnitRange{Int64}, Int64}, true}}, Nothing, Nothing, Nothing, Noth